# Using `spaCy` for text classification

Prior to running this code, it's necessary to install `spaCy` on your machine, and also to download its English libraries. 

*Note:* I'd like to install sklearn in the virtual environment, but `jupyter` is having trouble calling it. I've tried the following to troubleshoot, but so far no dice. Will solve this later.  
* https://stackoverflow.com/questions/42449814/running-jupyter-notebook-in-a-virtualenv-installed-sklearn-module-not-available  

In [13]:
import spacy
import pandas as pd
from spacy.tokens import Doc
from spacy.vocab import Vocab

In [14]:
# Load the pre-defined English model:
nlp = spacy.load('en')

### Example using Pandas dataframe

In [15]:
# Read in a CSV file with a column of text abstracts. Keep only the columns we need.
df = pd.read_csv('resources/fedreg_18-05-22-14-45.csv')
df = df[['agency', 'abstract', 'type']]
df = df.dropna(how='any') # get rid of missing data.
df.shape

(1411, 3)

In [16]:
# Let's do a little recording, and then take a look at the label column.
df.loc[df['type']=='Proposed Rule', 'type']='Rule'
df['type'].value_counts(dropna=False)

Notice    984
Rule      427
Name: type, dtype: int64

In [17]:
# Split into training and testing data. Keeping it small so the example doesn't take so long.
df_train=df.head(500)
df_test=df.tail(500)

In [18]:
# Create a dictionary with the abstract and its labels.
def dictionize_me(row):
    if row['type']=='Rule':
        return (row['abstract'], {"cats": {"Notice": 0, "Rule": 1}})
    else:
        return (row['abstract'], {"cats": {"Notice": 1, "Rule": 0}})

In [19]:
# Apply this function to the pandas dataframe.
df_train['newcol'] = df_train.apply(lambda x: dictionize_me(x), axis=1)
df_train.head(2)

/Users/austinlasseter/atelier/schoolhouse/learnspacy/spacyenv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,agency,abstract,type,newcol
1,Transportation Department,We are superseding Airworthiness Directive (AD...,Rule,(We are superseding Airworthiness Directive (A...
2,Commodity Futures Trading Commission,The Commodity Futures Trading Commission (Comm...,Rule,(The Commodity Futures Trading Commission (Com...


In [20]:
# Convert this into a list
train_data=list(df_train['newcol'])
train_data[0]

('We are superseding Airworthiness Directive (AD) 2017-11-03 for DG Flugzeugbau GmbH Model DG-500MB gliders that are equipped with a Solo 2625 02 engine modified with a fuel injection system following the instructions of Solo Kleinmoteren GmbH Technische Mitteilung 4600-3 and identified as Solo 2625 02i. This AD results from mandatory continuing airworthiness information (MCAI) issued by an aviation authority of another country to identify and correct an unsafe condition on an aviation product. The MCAI describes the unsafe condition as failure of the connecting rod bearing resulting from too much load on the rod bearings from the engine control unit. This AD adds a model to the applicability. We are issuing this AD to require actions to address the unsafe condition on these products.',
 {'cats': {'Notice': 0, 'Rule': 1}})

In [21]:
# Create an analytic "pipeline" of type "textcat"
mytextcat2 = nlp.create_pipe('textcat') # Note that we use 'textcat2' to distinguish from earlier example.
nlp.add_pipe(mytextcat2, last=True)

In [22]:
# Add labels to the pipeline. These will be called using the `.cats` attribute, below.
mytextcat2.add_label('Notice')
mytextcat2.add_label('Rule')

1

In [23]:
# Begin training. 
optimizer = nlp.begin_training()

In [24]:
# Apply to X_train
for itn in range(10):
    for doc, gold in train_data:
        nlp.update([doc], [gold], sgd=optimizer)

In [25]:
# Provide a new text, and classify it. The predicted category is called using the `.cats` attribute.
doc = nlp(u'TEST TEST TEST We are TEST superseding TEST Airworthiness Directive (AD) 2017-11-03 for DG TEST Flugzeugbau GmbH Model DG-500MB gliders that are equipped with a Solo 2625 02 engine modified with a fuel injection system following the instructions of Solo Kleinmoteren GmbH Technische Mitteilung 4600-3 and identified as Solo 2625 02i. This AD results from mandatory continuing airworthiness information (MCAI) issued by an aviation authority of another country to identify and correct an unsafe condition on an aviation product. The MCAI describes the unsafe condition as failure of the connecting rod bearing resulting from too much load on the rod bearings from the engine control unit. This AD adds a model to the applicability. TEST TEST TEST We are issuing this AD to require actions to address the unsafe condition on these products. TEST TEST TEST TEST')
print(doc.cats)

{'Notice': 4.539787187241018e-05, 'Rule': 0.9999257326126099}


In [70]:
# Classify the testing dataset.
def classify_testing_data(row):
    x=row.decode('utf-8')
    doc = nlp(x)
#     return doc.cats['Notice']
    return x

In [71]:
df_test['abstract'][1302]

'This notice sets forth the agenda of the forthcoming meeting of the National Museum and Library Services Board. This notice also describes the function of the Board. Notice of the meeting is required under the Sunshine in Government Act.'

In [72]:
classify_testing_data(df_test['abstract'][1302])

AttributeError: 'str' object has no attribute 'decode'

In [58]:
df_test['prob_notice']=df_test['abstract'].map(lambda x: classify_testing_data(x))

NameError: name 'x' is not defined